# Detect and save cropped image

In [ ]:
import os
import glob
import time

import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

CONFTHRESH = 0.5
NMSTHRESH = 0.3  # Need to try 0.3
PATH = './'

In [ ]:
def get_weights(weights_path):
    """
    Returns model.weights file path
    """
    return os.path.sep.join([PATH, weights_path])

def get_cfg(cfg_path):
    """
    Returns model.cfg file path
    """
    return os.path.sep.join([PATH, cfg_path])

def load_model(cfg_path, weights_path):
    """
    Returns loaded Darknet model
    """
    print("- Loading model")
    return cv2.dnn.readNetFromDarknet(cfg_path, weights_path)

def show_dif(img, crop):
    _, ax = plt.subplots(1, 2, figsize=(12, 5))
    ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax[0].set_title('Predicted image')
    ax[1].imshow(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
    ax[1].set_title('Cropped image')
    plt.show()

In [ ]:
def get_pred(image, model):
    """
    Returns prediction
    """
    (H, W) = image.shape[:2]
    layer_names = model.getLayerNames()
    layer_names = [
        layer_names[i[0] - 1] for i in model.getUnconnectedOutLayers()
    ]

    blob = cv2.dnn.blobFromImage(
        image,
        1 / 255.0,
        (416, 416),
        swapRB=True,
        crop=False
    )

    model.setInput(blob)
    layer_outputs = model.forward(layer_names)

    boxes = []
    confidences = []
    classes_ids = []

    for output in tqdm(layer_outputs, desc="- Making prediction"):

        for detected in output:

            scores = detected[5:]
            class_id = np.argmax(scores)
            conf = scores[class_id]

            if conf > CONFTHRESH:
                box = detected[0:4] * np.array([W, H, W, H])
                (x_center, y_center, width, height) = box.astype("int")

                x = int(x_center - (width / 2))
                y = int(y_center - (height / 2))

                boxes.append(
                    [x, y, int(width), int(height)]
                )
                confidences.append(float(conf))
                classes_ids.append(class_id)

    idxs = cv2.dnn.NMSBoxes(
        boxes,
        confidences,
        CONFTHRESH,
        NMSTHRESH
    )

    if len(idxs) > 0:

        for i in idxs.flatten():

            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            cropped_image = image[y:y+h, x:x+w]

    return cropped_image


def main(cfg, weights, filename, output, batch=False):

    cfg = get_cfg(cfg)
    weights = get_weights(weights)
    model = load_model(cfg, weights)
    
    if batch:
        for i in glob.glob(os.path.join(filename, '*.jpg')):
            image = cv2.imread(i, cv2.IMREAD_UNCHANGED)

            start = time.time()
            output_image = get_pred(
                image=image,
                model=model,
            )
            print(f" Predicted {i} in {time.time() - start:.2f} seconds")
            out_img_path = os.path.join(
                output,
                os.path.basename(i)
            )
            output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)
            cv2.imwrite(out_img_path, output_image)
            print(f"- Saved predicted image to {out_img_path}")
            show_dif(image, output_image)
    else:
        image = cv2.imread(filename, cv2.IMREAD_UNCHANGED)

        start = time.time()
        output_image = get_pred(
            image=image,
            model=model,
        )
        print(f"- Predicted {filename} in {time.time() - start:.2f} seconds")
        out_img_path = os.path.join(
            output,
            os.path.basename(filename)
        )
        output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)
        cv2.imwrite(out_img_path, output_image)
        print(f"- Saved predicted image to {out_img_path}")
        show_dif(image, output_image)

In [ ]:
main(
    cfg='cfg/custom/tiny-armoire.cfg',
    weights='backup/tiny-armoire_last.weights',
    filename='samples',
    output='predictions/',
    batch=True
)